# NER
В этом документе я создаю теги для нужных нам сущностей, обучаю модель распознавания на них и вывожу результат этого распознавания


В этой ячейке проверяем, как работает модель spaCy для русского языка. Поскольку сейчас исопльзуется предобученная на датасете новостей модель, она умеет определять только формальные сущности (персона, локация, время), которые неприменимы к нашему проекту. Поэтому далее я буду создавать распознаватель конкретно под наши задачи. 

А в этой ячейке удостоверяемся, что spaCy корректно отвечает на заданные для нее условия (определила "Ульяна" как "персону", "Москве" как "локацию"

In [155]:
import spacy
import pandas as pd
import random
import json
from spacy.lang.ru import Russian
from spacy.pipeline import EntityRuler

import ru_core_news_lg


nlp = ru_core_news_lg.load()
test = 'ульяна в москве'

doc = nlp(test)
for ent in doc.ents:    
    print(ent.text, ent.label_)

ульяна PER
москве LOC


Считываем датасет всех команд, далее выбираем отдельно столбец с командами для дальнейшей работы

In [2]:
import pandas as pd
all_df = pd.read_csv('all_commands.csv')
all_commands = pd.read_csv('all_commands.csv')[['command']]

In [3]:
all_df

,Unnamed: 0,command,intent,entity
0,0,я быть в отчаяние чтобы пойти направо,move_ship_by_direction,ship_direction
1,1,я пойти наверх корабль на,move_ship_by_direction,ship_direction
2,2,давать подняться наверх,move_ship_by_direction,ship_direction
3,3,пожалуйста слева слышать,move_ship_by_direction,ship_direction
4,4,пойти я на корабль,move_ship_by_direction,ship_direction
...,...,...,...,...
11109,11109,возвращаться к свой пиратка,pirate_rebirth,none
11110,11110,я хотеть крепость в отдохнуть,pirate_rebirth,none
11111,11111,я хотеть отдохнуть в крепость,pirate_rebirth,none
11112,11112,крепость хотеть отдохнуть в я,pirate_rebirth,none


Переводим колонку команд в список

In [157]:
all_commands_list = []
for i in all_commands['command']:
        all_commands_list.append(i)

Функция для записи списка команд в JSON файл, с ним далее удобнее работать

In [158]:
def write_list(a_list):
    with open("all_commands.json", "w", encoding='utf8') as f:
        json.dump(a_list, f, ensure_ascii=False)

write_list(all_commands_list)

Заводим 2 функции для чтения файла и сохранения в файл. Загружаем из JSON, сохраняем тоже в JSON

In [159]:
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

def save_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

Пишем функции для обработки команд. 

In [160]:
# В отдельном файле у нас есть список сущностей и их лэйбл. Библиотека spaCy требует особой 
# структуры данных для обработки пар сущность — лэйбл (это паттерн). Эта функция позволяет создать такую структуру
# из имеющихся данных 

def create_training_data(file, type):
    data = load_data(file)
    patterns = []
    for item in data:
        pattern = {
                    "label": type,
                    "pattern": item
                    }
        patterns.append(pattern)
    return (patterns)



# Эта фунция создает и сохраняет кастомную модель NER, которая работает с созданными выше паттернами.

def generate_rules(patterns):
    nlp = Russian()
    ruler = nlp.add_pipe("entity_ruler")
    ruler.add_patterns(patterns)
    nlp.to_disk("jackal_ner_all_entities")
    

    
# Эта функция обрабатывает входящий текст (ищет сущности) с использованием созданной выше модели и записывает
# найденное в список 
    
def test_model(model, text):
    doc = nlp(text)
    results = []
    entities = []
    for ent in doc.ents:
        entities.append((ent.start_char, ent.end_char, ent.label_))
    if len(entities) > 0:
        results = [text, {"entities": entities}] # специальный формат для spaCy
        return (results)
                

                                                                            

Создаем паттерны для каждого типа сущностей и объединяем в единый список — это нужно, чтобы конечная модель имела в себе паттерны сущностей всех типов

In [161]:
patterns_dir = create_training_data("NER_dir.json", "DIR")
patterns_tile = create_training_data("NER_tiles.json", "TILE")
pattern_act = create_training_data("NER_act.json", "ACT")
pattern_num = create_training_data("NER_num.json", "NUM")

all_patterns = patterns_dir + patterns_tile + pattern_act + pattern_num

Создаем модель распознавания сущностей

In [162]:
generate_rules(all_patterns)

# Объединила паттерны и сделала единую модель, чтобы иметь множество лейблов, а не один
# print (patterns)

Функция ниже создает список команд из JSON файла, а далее создает тренировочный размеченный датасет в 70% от всего объема команд

In [163]:
def get_text(file):
    data = load_data(file)
    text = []
    for item in data:
        text.append(item) 
    return (text)


nlp = spacy.load("jackal_ner_all_entities")
TRAIN_DATA = []
outsiders = []
text = get_text("all_commands.json")
hits = []
counter = 0
test_size = round(0.7 * len(text))
while counter < len(text[0:test_size]): # делаем тренировочный датасет в 70% от всего
    for command in text:
        command = command.strip()
    #        command = command.replace("\n", " ")
        results = test_model(nlp, command)
        if results != None:
            TRAIN_DATA.append(results)
            
        ########################
        #далее идут временный команды. нужны для отслеживания качества
        else:
             outsiders.append(command)
            
        
        #######################
        counter += 1

Сохраняем тренировочный датасет

In [164]:
save_data("ML_NER_train_data.json", TRAIN_DATA)

Заметим, что не все сущности были распознаны, TODO НАДО УЗНАТЬ В ЧЕМ ПРОБЛЕМА

In [165]:
print(1 - len(TRAIN_DATA)/len(text))

# заметим по ячейкам ниже, что "None" доолжен быть в 216/11,113 случаях, а у меня получилось, что в 30%
# то есть пока % остаются нераспознанными, надо выяснить причину

0.09546517905344609


Выведем те команды, в которых ничего не было распознано

In [166]:
nothing_count = len(all_df[all_df["intent"]=="nothing"])
out_count = len(outsiders)
print(f"Всего таких: {out_count}, а в колонке 'nothing' только {nothing_count}\n\
Значит проблемы с определением сущности в {out_count-nothing_count} случаях")

Всего таких: 1061, а в колонке 'nothing' только 216
Значит проблемы с определением сущности в 845 случаях


В этой ячейке фиксирую прогресс улучшений распознавания сущностей:

**Сразу после генераяии:**
Всего таких: 3141, а в колонке 'nothing' только 216
Значит проблемы с определением сущности в 2925 случаях

**Добавила "левый", "правый", "верх", "низ" в направления, "судно" в действующие лица:** 
Всего таких: 2621, а в колонке 'nothing' только 216
Значит проблемы с определением сущности в 2405 случаях

**Добавила "ниже" в направления, "лодка", "борт" в действующие лица, "монета", "золото", "деньга" в карточки:**  Всего таких: 2054, а в колонке 'nothing' только 216
Значит проблемы с определением сущности в 1838 случаях

**Добавила "" в направления, "самолёт", "палуба", "штурвал" в действующие лица, "сокровище", "сокровищница", "тропик","золотишко", "пустынь", "заросль", "лёд", "гора", "скала" , "лес", "девушка", "абориген"  в карточки:**  Всего таких: 1341, а в колонке 'nothing' только 216
Значит проблемы с определением сущности в 1125 случаях


**Добавила "наверху", "внизу"  в направления, "пиратка", "", "" в действующие лица, "песок", "дюна", "клетка","сундучок", "мелочь", "клеточка", "кораблик", "аборигенка", "монетка" , "холм", "клад", "лайнер", "тропический", "леса"  в карточки:**  
Всего таких: 1061, а в колонке 'nothing' только 216
Значит проблемы с определением сущности в 845 случаях


In [167]:
import random
random.shuffle(outsiders)
outsiders

['по что у мы выяснить диагональ',
 'идти я далёкий',
 'подняться давать',
 'очень спешить и туда обратно',
 'я идти любимый к леда',
 'давать вернуться давать',
 'отсюда возвращаться я',
 'я пойти низкий',
 'давать подняться',
 'я пойти на кладбище',
 'я кролик с пойти',
 'быстро к полянин',
 'пэстера забирать свой я',
 'я уплывать отсюда',
 'курсор к любимый перейти',
 'я правда плыть',
 'я взять пицца',
 'я уходить из подвал',
 'мы плыть далёкий',
 'мотор заводить мы спускаться',
 'гребец пиратский вернуться в россия',
 'пиастр я забирать',
 'всё ровер ещё грестит',
 'по следовать вода',
 'морепродукт хотеть я',
 'уходить я',
 'быстрый море',
 'пиастр я уносить',
 'просить прощение чувак',
 'я леда к вернуться',
 'полянин на потопаться',
 'любимый идти к я джунгнуть',
 'что ты сделать',
 'хороший каннибал к',
 'я спускаться тропин по',
 'парить я над кролик',
 'так и есть',
 'осколок нравиться я',
 'брас хороший',
 'хороший спешить очень',
 'я возвращаться',
 'вы ошибаться',
 'мы ид

In [168]:
nothing_list = all_df[all_df["intent"]=="nothing"][['command']].to_numpy()

In [169]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

print (outsiders not in intersection(outsiders, nothing_list))
print (len(intersection(outsiders, nothing_list)))

True
83


Из ячейки ниже заметим, что сущностей нет только в 216 строках — это колонка "nothing"

In [170]:
print(all_df.groupby(['intent']).count())

                          Unnamed: 0  command  entity
intent                                               
board_ship                       167      167     167
hit_enemy_pirate                 155      155     155
leave_coin                       195      195     195
leave_ship                       102      102     102
move_in_labyrinth                377      377     377
move_pirate_by_direction        2734     2734    2734
move_pirate_by_tile             2902     2902    2902
move_ship_by_direction          2329     2329    2329
nothing                          216      216     216
pirate_rebirth                   213      213     213
pirate_swim                     1523     1523    1523
take_coin                        201      201     201


In [171]:
print(len(all_df[all_df["intent"]=="nothing"]))

216


In [172]:
results = test_model(nlp, 'первым пиратом пойти направо на шар')
results

results_2 = test_model(nlp, 'первый пират пойти направо на шар')
results_2


# "пиратом" не видит, хотя в сущностях есть слово "пират". теперь надо понять, 
# что для этого сделать — добавить все окончания в список сущностей, или на входе лемматизировать слова в команде и уже по новой искать

['первый пират пойти направо на шар',
 {'entities': [(0, 6, 'NUM'),
   (7, 12, 'ACT'),
   (19, 26, 'DIR'),
   (30, 33, 'TILE')]}]

## Тренировка модели

In [ ]:
import random
from spacy.training.example import Example

def train_spacy(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.blank("ru")
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner")
    else:
        ner = nlp.get_pipe("ner")
    
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.select_pipes(disable=other_pipes):
        optimizer = nlp.initialize()
        for epoch in range(epochs)
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                example = Example.from_dict(nlp.make_doc(text), annotations)
                nlp.update(
                    [example],
                    sgd=optimizer,
                    losses=losses
                )
            print(f"Epoch {epoch}, losses {losses}")
    return nlp

In [ ]:
TRAIN_DATA = load_data("ML_NER_train_data.json")

nlp = train_spacy(TRAIN_DATA, 30)
nlp.to_disk("jackal_ner_trained_model")

In [5]:
test = 'первым пиратом пойду налево и попаду на шар'
import re
import spacy
import nltk
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text




test = preprocess_text(test)
people = []
nlp = spacy.load("jackal_ner_all_entities")
doc = nlp(test)
for ent in doc.ents:
    print (ent.text, ent.label_)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pabakst/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


первый NUM
пират ACT
налево DIR
шар TILE
